In [3]:
import pinocchio as pin
import pinocchio.casadi as cpin
import casadi as cs
from pinocchio.robot_wrapper import RobotWrapper
import sys, os
import numpy as np
from enum import Enum
from pathlib import Path

import os, sys
from symbolic_generator import SymbolicGenerator, KinematicsOrientation, cpin, cs
import math

In [4]:
urdf_path = '../models/nadia/nadiaV17.fullRobot.simpleKnees.fixedArms_mj.urdf'
mesh_dir = "../models/nadia"
symb_gen = SymbolicGenerator(urdf_path, mesh_dir=mesh_dir,
                             floating = True,
                             kinematics_bodies=['L_C', 'R_C'],
                             kinematics_ori = KinematicsOrientation.AxisAngle,
                             actuated_dofs = slice(6,21))
symb_gen.generate()
model = RobotWrapper.BuildFromURDF(urdf_path, mesh_dir, root_joint = pin.JointModelFreeFlyer()).model
data = model.createData()
cmodel = cpin.Model(model)
cdata = cmodel.createData()


Loaded robot model from: ../models/nadia/nadiaV17.fullRobot.simpleKnees.fixedArms_mj.urdf
----------- Model Details -----------
	Floating base = true
	# of configs = 22
	# of DoFs = 21

	Bodies: ['world', 'pelvis', 'PELVIS_LINK', 'LEFT_HIP_Z', 'LEFT_HIP_YAW_LINK', 'LEFT_HIP_X', 'LEFT_HIP_ROLL_LINK', 'LEFT_HIP_Y', 'LEFT_THIGH_LINK', 'LEFT_KNEE_Y', 'LEFT_SHIN_LINK', 'LEFT_ANKLE_Y', 'LEFT_ANKLE_LINK', 'LEFT_ANKLE_X', 'LEFT_FOOT_LINK', 'L_C_joint', 'L_C', 'L_FL_joint', 'L_FL', 'L_FR_joint', 'L_FR', 'L_RL_joint', 'L_RL', 'L_RR_joint', 'L_RR', 'LEFT_SHIN_IMU_JOINT', 'LEFT_SHIN_IMU_LINK', 'LEFT_THIGH_IMU_JOINT', 'LEFT_THIGH_IMU_LINK', 'PELVIS_STIM_IMU_JOINT', 'PELVIS_STIM_IMU_LINK', 'RIGHT_HIP_Z', 'RIGHT_HIP_YAW_LINK', 'RIGHT_HIP_X', 'RIGHT_HIP_ROLL_LINK', 'RIGHT_HIP_Y', 'RIGHT_THIGH_LINK', 'RIGHT_KNEE_Y', 'RIGHT_SHIN_LINK', 'RIGHT_ANKLE_Y', 'RIGHT_ANKLE_LINK', 'RIGHT_ANKLE_X', 'RIGHT_FOOT_LINK', 'R_C_joint', 'R_C', 'R_FL_joint', 'R_FL', 'R_FR_joint', 'R_FR', 'R_RL_joint', 'R_RL', 'R_RR_joint

In [73]:
nq, nv = model.nq, model.nv
q = pin.neutral(model)
v = np.zeros(nv)
v[3] = 1
q[3:7] = pin.exp3_quat(np.array([0, math.pi, 0]))

print(q)
print(v)

pin.forwardKinematics(model, data, q, v)
pin.updateFramePlacements(model, data)

# id = model.getFrameId('R_C')
id = 1

test = pin.getFrameVelocity(model, data, id, pin.ReferenceFrame.LOCAL_WORLD_ALIGNED)

print(test.angular)

x_k_sym = cs.SX.sym('x_k_sym', nq + nv) 
cpin.forwardKinematics(cmodel, cdata, x_k_sym[:nq])
cpin.updateFramePlacements(cmodel, cdata)
kinematics = []
kinematics.append(cdata.oMf[id].translation)
aa = cpin.log3(cdata.oMf[id].rotation)
kinematics.append(aa)
kinematics = cs.vertcat(*kinematics)
E = cs.substitute(cs.densify(cs.jacobian(cpin.integrate(cmodel, x_k_sym[:nq], x_k_sym[nq:]), x_k_sym[nq:])), x_k_sym[nq:], cs.SX.zeros(nv))
kinematics_dot = cs.jacobian(kinematics, x_k_sym)[:, :nq]@E@x_k_sym[nq:]
kinematics_dot_func = cs.Function('kinematics_dot', [x_k_sym], [kinematics_dot])
E_func = cs.Function('E', [x_k_sym], [(cs.jacobian(kinematics, x_k_sym[:nq])@E)[:, :7]])
np.set_printoptions(precision=2, suppress=True, linewidth=100)
print(np.array(kinematics_dot_func(np.concatenate([q, v]))).flatten())
print(np.array(E_func(np.concatenate([q, v]))))

[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[-1.  0. -0.]
[ 0.    0.    0.    0.    0.   -1.57]
[[-1.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    1.    0.    0.    0.    0.    0.  ]
 [-0.    0.   -1.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    1.57  0.  ]
 [ 0.    0.    0.    0.    1.    0.    0.  ]
 [ 0.    0.    0.   -1.57  0.    0.    0.  ]]


In [68]:
x_k_sym = cs.SX.sym('x_k_sym', nq + nv) 
cpin.forwardKinematics(cmodel, cdata, x_k_sym[:nq])
cpin.updateFramePlacements(cmodel, cdata)
kinematics = []

for body in symb_gen.kinematics_bodies:
    kinematics.append(cdata.oMf[cmodel.getFrameId(body)].translation)
    aa = cpin.log3(cdata.oMf[cmodel.getFrameId(body)].rotation)
    kinematics.append(aa)
kinematics = cs.vertcat(*kinematics)
E = cs.substitute(cs.densify(cs.jacobian(cpin.integrate(cmodel, x_k_sym[:nq], x_k_sym[nq:]), x_k_sym[nq:])), x_k_sym[nq:], cs.SX.zeros(nv))
kinematics_dot = cs.jacobian(kinematics, x_k_sym)[:, :nq]@E@x_k_sym[nq:]
kinematics_dot_func = cs.Function('kinematics_dot', [x_k_sym], [kinematics_dot])

In [69]:
np.array(kinematics_dot_func(np.concatenate([q, v])))[9:]
np.linalg.norm(np.array(kinematics_dot_func(np.concatenate([q, v])))[9:])

5.651921387810258

In [74]:
nq, nv = model.nq, model.nv
q = pin.neutral(model)
v = np.zeros(nv)
v[3] = 1
q[3:7] = pin.exp3_quat(np.array([0, math.pi, 0]))

print(q)
print(v)

pin.forwardKinematics(model, data, q, v)
pin.updateFramePlacements(model, data)

# id = model.getFrameId('R_C')
id = 1

test = pin.getFrameVelocity(model, data, id, pin.ReferenceFrame.LOCAL_WORLD_ALIGNED)
print(test.angular)

def skew_symmetric(v):
    """Create a skew-symmetric matrix from a 3-element vector."""
    return np.array([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])

quat = cs.SX.sym('quat', 4)
w = cs.SX.sym('w', 3)
a = cpin.log3(quat)
da_dq = cs.jacobian(a, quat)
G = cs.vertcat(-cs.horzcat(quat[0], quat[1], quat[2]), quat[3]*cs.SX.eye(3) + skew_symmetric(quat[:3]))
dq_dt = 0.5*G@w
da_dt = da_dq@dq_dt
print(da_dt)
test_func = cs.Function('da_dt', [quat, w], [da_dt])
test_func(q[3:7], v[3:6])

[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[-1.  0. -0.]
@1=(0<=quat_3), @2=1, @3=((@1?@2:0)+((!@1)?-1:0)), @4=(@3*quat_0), @5=(sq(quat_0)+(sq(quat_1)+sq(quat_2))), @6=6.05545e-06, @7=(@5<@6), @8=2, @9=0.166667, @10=0.25, @11=(@5<@6), @12=(@3*quat_3), @13=sq(@12), @14=(@5/@13), @15=(@8*(@2-(@14/3))), @16=(@5+4.93038e-32), @17=sqrt(@16), @18=(@17/@12), @19=(!@11), @20=atan2(@17,@12), @21=((@11?(@15*@18):0)+(@19?(@8*@20):0)), @22=(@21+@21), @23=(quat_1/@17), @24=0.333333, @25=(@16+sq(@12)), @26=(@12/@25), @27=(@26*@23), @28=((@11?((@15*(@23/@12))-(@18*(@8*(@24*((quat_1+quat_1)/@13))))):0)+(@19?(@8*@27):0)), @29=(@10*(@22*@28)), @30=(sq(@21)/4), @31=0.0194444, @32=(@31*@30), @33=(!@7), @34=sin(@20), @35=(@21/@34), @36=(@35/@34), @37=cos(@20), @38=((@7?(@8*((@9*@29)+((@30*(@31*@29))+(@32*@29)))):0)+(@33?((@28/@34)-(@36*(@37*@27))):0)), @39=0.5, @40=((((@39*quat_3)*w_0)-((@39*quat_2)*w_1))+((@39*quat_1

DM([0, 1, 0])